In [1]:
# How many images in each folder?
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/c

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

target_shape = (224, 224)
batch_size = 8

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               target_size=target_shape,
                                               batch_size=batch_size,
                                               class_mode='categorical')

test_data = test_datagen.flow_from_directory(directory=test_dir,
                                               target_size=target_shape,
                                               batch_size=batch_size,
                                               class_mode='categorical')


Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [25]:
import datetime

def tensorboard_callback(dir_name, experiment_name):
    log_dir = f'{dir_name}/experiment_name{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tb_callback


In [14]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers


In [15]:
# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Original: EfficientNetB0 feature vector (version 1)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# # New: EfficientNetB0 feature vector (version 2)
# efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

In [22]:
def create_model(model_url, num_classes=10):

    feature_extractor = hub.KerasLayer(model_url, 
                                       trainable=False,
                                       name='feature_extractor_layer',
                                       input_shape=target_shape+(3,))
    
    inputs = tf.keras.layers.Input(shape=(target_shape+(3,)))
    x = feature_extractor(inputs)
    x = tf.keras.layers.Dense(num_classes, activation='softmax', name='classifier')(x)
    return tf.keras.Model(inputs=inputs, outputs=x)
model = create_model(resnet_url)
model.summary()
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
feature_extractor_layer (Ker (None, 2048)              23564800  
_________________________________________________________________
classifier (Dense)           (None, 10)                20490     
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


In [27]:
resnet_history = model.fit(train_data, 
                           epochs=5,
                           steps_per_epoch=len(train_data),
                           validation_data=test_data,
                           validation_steps=len(test_data),
                           callbacks=[tensorboard_callback(dir_name='tf_hub', experiment_name='resnet50')]
                           )

2021-11-11 10:34:09.933987: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-11-11 10:34:09.934013: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-11-11 10:34:09.939205: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2021-11-11 10:34:09.939296: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-11-11 10:34:09.939314: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1752] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.


Saving TensorBoard log files to: tf_hub/experiment_name20211111-103409
Epoch 1/5
 1/94 [..............................] - ETA: 32s - loss: 0.1514 - accuracy: 1.0000

2021-11-11 10:34:10.582965: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-11-11 10:34:10.583010: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-11-11 10:34:10.583130: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.


 2/94 [..............................] - ETA: 44s - loss: 0.1260 - accuracy: 1.0000

2021-11-11 10:34:10.895864: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-11 10:34:10.895970: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1752] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2021-11-11 10:34:10.896615: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 0 callback api events and 0 activity events. 
2021-11-11 10:34:10.899585: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-11-11 10:34:10.901855: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: tf_hub/experiment_name20211111-103409/train/plugins/profile/2021_11_11_10_34_10
2021-11-11 10:34:10.902994: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to tf_hub/experiment_name20211111-103409/train/plugins/profile/2021_11_11_10_34_10/viktor-

94/94 [==============================] - 107s 1s/step - loss: 0.1365 - accuracy: 0.9853 - val_loss: 0.6259 - val_accuracy: 0.7976
Epoch 2/5
94/94 [==============================] - 105s 1s/step - loss: 0.1048 - accuracy: 0.9947 - val_loss: 0.6147 - val_accuracy: 0.7972
Epoch 3/5
94/94 [==============================] - 109s 1s/step - loss: 0.0828 - accuracy: 0.9960 - val_loss: 0.6455 - val_accuracy: 0.7848
Epoch 4/5
94/94 [==============================] - 108s 1s/step - loss: 0.0665 - accuracy: 0.9973 - val_loss: 0.6375 - val_accuracy: 0.7912
Epoch 5/5
94/94 [==============================] - 103s 1s/step - loss: 0.0549 - accuracy: 0.9987 - val_loss: 0.6303 - val_accuracy: 0.7964


In [30]:
efficient_net_model = create_model(model_url=efficientnet_url)

efficient_net_model.summary()

efficient_net_model.compile(loss='categorical_crossentropy',
                            optimizer=tf.keras.optimizers.Adam(),
                            metrics=['accuracy'])


efficient_net_model.fit(train_data,
                        epochs=5,
                        steps_per_epoch=len(train_data),
                        #validation_data=test_data,
                        #validation_steps=len(test_data),
                        callbacks=[tensorboard_callback(dir_name='tf_hub', 
                                        experiment_name='efficient_net')])

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
feature_extractor_layer (Ker (None, 1280)              4049564   
_________________________________________________________________
classifier (Dense)           (None, 10)                12810     
Total params: 4,062,374
Trainable params: 12,810
Non-trainable params: 4,049,564
_________________________________________________________________
Saving TensorBoard log files to: tf_hub/experiment_name20211111-104334
Epoch 1/5


2021-11-11 10:43:34.399484: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-11-11 10:43:34.399505: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-11-11 10:43:34.399565: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2021-11-11 10:43:34.399684: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-11-11 10:43:34.399697: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1752] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.


 2/94 [..............................] - ETA: 1:18 - loss: 2.4202 - accuracy: 0.1250    

2021-11-11 10:43:40.925985: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-11-11 10:43:40.926010: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-11-11 10:43:40.926089: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.


 3/94 [..............................] - ETA: 1:02 - loss: 2.4983 - accuracy: 0.1250

2021-11-11 10:43:41.441434: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-11 10:43:41.441559: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1752] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2021-11-11 10:43:41.443688: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 0 callback api events and 0 activity events. 
2021-11-11 10:43:41.446351: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-11-11 10:43:41.451726: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: tf_hub/experiment_name20211111-104334/train/plugins/profile/2021_11_11_10_43_41
2021-11-11 10:43:41.455114: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to tf_hub/experiment_name20211111-104334/train/plugins/profile/2021_11_11_10_43_41/viktor-

94/94 [==============================] - 24s 195ms/step - loss: 1.4598 - accuracy: 0.5600
Epoch 2/5
94/94 [==============================] - 14s 148ms/step - loss: 0.6830 - accuracy: 0.8160
Epoch 3/5
94/94 [==============================] - 15s 157ms/step - loss: 0.4830 - accuracy: 0.8947
Epoch 4/5
94/94 [==============================] - 15s 160ms/step - loss: 0.3745 - accuracy: 0.9267
Epoch 5/5
94/94 [==============================] - 14s 149ms/step - loss: 0.2962 - accuracy: 0.9507


In [34]:
# Upload to Tensorboard

!tensorboard dev upload --logdir tf_hub \
    --one_shot \
    --name "resnet50 vs efficient net" \
    --description "Comparing Resnet50 with the Efficient net"

2021-11-11 10:45:37.122946: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-11 10:45:37.122964: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

tf_hub

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) ^C
Tracebac

In [35]:
# Check out experiments
!tensorboard dev list

2021-11-11 10:47:38.571987: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-11 10:47:38.572004: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
https://tensorboard.dev/experiment/tR81OZU3T2aBfDd5VGCQCQ/
	Name                 resnet50 vs efficient net
	Description          Comparing Resnet50 with the Efficient net
	Id                   tR81OZU3T2aBfDd5VGCQCQ
	Created              2021-11-11 10:46:24 (1 minute ago)
	Updated              2021-11-11 10:47:00 (48 seconds ago)
	Runs                 5
	Tags                 5
	Scalars              64
	Tensor bytes         0
	Binary object bytes  7975580
Total: 1 experiment(s)
